# NFL Player SportsIQ Rating System

This notebook implements a position-specific player rating system (SportsIQ Score) that evaluates NFL players on a 0-100 scale using weighted z-scores. The system:

- **Loads comprehensive NFL data** including rosters, snap counts, player stats, and advanced metrics from the 2025 season via nflreadpy
- **Implements position-specific scoring classes** (QB, RB, WR, TE, DL, LB) that use configurable weighted metrics tailored to each position's key performance indicators
- **Calculates normalized ratings** by computing z-scores against positional peers, applying directional weights (higher/lower is better), and converting to a T-score scale (0-100)
- **Filters players** based on minimum playing time thresholds (e.g., QBs: 50 attempts, RBs: 50 carries, WRs: 30 targets)
- **Exports detailed breakdowns** including raw metrics, z-scores, weighted contributions, and final SportsIQ ratings to CSV files

The scoring methodology emphasizes efficiency metrics (EPA, CPOE, YAC) over raw volume stats, with position-specific weights reflecting the relative importance of each metric (e.g., QB sacks are heavily penalized, DL pressure rate is highly valued).

In [246]:
import nflreadpy as nfl
import pandas as pd
import numpy as np
from abc import ABC, abstractmethod
from typing import Dict
import os
from datetime import datetime, timedelta

In [247]:
def load_df_master_with_cache(season: int, cache_file: str, cache_hours=24):
    """
    Load df_master from cache if available and recent, otherwise regenerate.
    
    Args:
        cache_file: Name of CSV file to cache to
        cache_hours: Number of hours before cache expires (default: 24)
    
    Returns:
        df_master: The loaded/generated DataFrame
        from_cache: Boolean indicating if data came from cache
    """
    print(f"\nChecking for cached df_master in '{cache_file}'...")

    # Check if cache file exists and is fresh
    if os.path.exists(cache_file):
        file_age = datetime.now() - datetime.fromtimestamp(os.path.getmtime(cache_file))
        if file_age < timedelta(hours=cache_hours):
            print(f"✓ Loading from cache: {cache_file}")
            print(f"  Cache age: {file_age.total_seconds() / 3600:.1f} hours")
            return pd.read_csv(cache_file), True
        else:
            print(f"⊘ Cache expired ({file_age.total_seconds() / 3600:.1f} hours old), regenerating...")
    else:
        print(f"⊘ Cache file not found, generating new data...")
    
    # Regenerate data
    print("\nLoading data from NFL API...")
    
    # 1. Load Rosters
    roster = nfl.load_rosters(seasons=[season]).to_pandas()
    roster_key = roster[['full_name', 'position', 'team', 'gsis_id', 'pfr_id', 'depth_chart_position']]
    roster_key = roster_key[roster_key['gsis_id'].notna()]
    roster_key = roster_key[roster_key['pfr_id'].notna()]
    print("  ✓ Roster loaded")
    
    # 2. Load Snap Counts
    snaps = nfl.load_snap_counts(seasons=[season]).to_pandas()
    snaps_season = snaps.groupby('pfr_player_id')[['offense_snaps', 'defense_snaps', 'st_snaps']].sum().reset_index()
    # Count unique games played per player
    games_played = snaps.groupby('pfr_player_id')['week'].nunique().reset_index().rename(columns={'week': 'games_played'})
    snaps_season = snaps_season.merge(games_played, on='pfr_player_id', how='left')
    print("  ✓ Snap counts loaded")
    
    # 3. Load Player Stats
    player_stats = nfl.load_player_stats(seasons=[season], summary_level='reg').to_pandas()
    adv_def = nfl.load_pfr_advstats(seasons=[season], summary_level='season', stat_type='def').to_pandas()
    adv_pass = nfl.load_pfr_advstats(seasons=[season], summary_level='season', stat_type='pass').to_pandas()
    adv_rush = nfl.load_pfr_advstats(seasons=[season], summary_level='season', stat_type='rush').to_pandas()
    adv_rec = nfl.load_pfr_advstats(seasons=[season], summary_level='season', stat_type='rec').to_pandas()
    print("  ✓ Player stats loaded")
    
    # 4. Merge all data
    df_master = roster_key.copy()
    df_master = df_master.merge(snaps_season, left_on='pfr_id', right_on='pfr_player_id', how='left')
    df_master = df_master.merge(player_stats, left_on='gsis_id', right_on='player_id', how='left')
    df_master = df_master.merge(adv_def, on='pfr_id', how='left', suffixes=('', '_def'))
    df_master = df_master.merge(adv_pass, on='pfr_id', how='left', suffixes=('', '_pass'))
    df_master = df_master.merge(adv_rush, on='pfr_id', how='left', suffixes=('', '_rush'))
    df_master = df_master.merge(adv_rec, on='pfr_id', how='left', suffixes=('', '_rec'))
    print("  ✓ Data merged")
    
    # 5. Clean up
    df_master = df_master.drop_duplicates(subset=['gsis_id'], keep='first')
    df_master = df_master[(df_master['offense_snaps'] > 0) | (df_master['defense_snaps'] > 0) | (df_master['st_snaps'] > 0)]
    df_master = df_master.dropna(subset=['offense_snaps', 'defense_snaps', 'st_snaps'])
    print("  ✓ Data cleaned")
    
    # Save to cache
    df_master.to_csv(cache_file, index=False)
    print(f"✓ Data saved to cache: {cache_file}")
    
    return df_master, False



In [248]:
# Load or generate df_master
season = 2025
file = f'nfl_player_master_{season}.csv'

df_master, from_cache = load_df_master_with_cache(season=season, cache_file=file)


Checking for cached df_master in 'nfl_player_master_2025.csv'...
✓ Loading from cache: nfl_player_master_2025.csv
  Cache age: 1.2 hours


In [249]:
# 4.b Deduplicate by unique player (gsis_id), keeping first occurrence
df_master = df_master.drop_duplicates(subset=['gsis_id'], keep='first')

# Remove players with 0 snaps or null values
df_master = df_master[(df_master['offense_snaps'] > 0) | (df_master['defense_snaps'] > 0) | (df_master['st_snaps'] > 0)]
df_master = df_master.dropna(subset=['offense_snaps', 'defense_snaps', 'st_snaps'])

In [250]:
# Check for duplicates
print("Checking for duplicates by full_name:")
duplicate_names = df_master['full_name'].value_counts()
duplicates = duplicate_names[duplicate_names > 1]
print(f"\nFound {len(duplicates)} players with multiple rows:")
print(duplicates)

# Show Joe Flacco example
print("\n\nJoe Flacco rows:")
print(df_master[df_master['full_name'] == 'Joe Flacco'][['full_name', 'gsis_id', 'pfr_id', 'team', 'depth_chart_position']])

print("\n\nColumn names:")
print(df_master.columns.tolist())

Checking for duplicates by full_name:

Found 3 players with multiple rows:
full_name
Jordan Phillips    2
Jaylon Jones       2
Byron Young        2
Name: count, dtype: int64


Joe Flacco rows:
    full_name     gsis_id    pfr_id team depth_chart_position
5  Joe Flacco  00-0026158  FlacJo00  CIN                   QB


Column names:
['full_name', 'position_x', 'team', 'gsis_id', 'pfr_id', 'depth_chart_position', 'pfr_player_id', 'offense_snaps', 'defense_snaps', 'st_snaps', 'games_played', 'player_id', 'player_name', 'player_display_name', 'position_y', 'position_group', 'headshot_url', 'season', 'season_type', 'recent_team', 'games', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 

In [251]:
# Examine remaining duplicates to understand if they're legitimate
print("Remaining duplicates - checking if they're different players:\n")
for dup_name in ['Jordan Phillips', 'Jaylon Jones', 'Byron Young']:
    print(f"\n{dup_name}:")
    dup_rows = df_master[df_master['full_name'] == dup_name][['full_name', 'gsis_id', 'pfr_id', 'team', 'position_x', 'depth_chart_position']]
    print(dup_rows)

Remaining duplicates - checking if they're different players:


Jordan Phillips:
            full_name     gsis_id    pfr_id team position_x  \
68    Jordan Phillips  00-0031557  PhilJo01  BUF         DL   
1578  Jordan Phillips  00-0040175  PhilJo02  MIA         DL   

     depth_chart_position  
68                     DT  
1578                   DT  

Jaylon Jones:
         full_name     gsis_id    pfr_id team position_x depth_chart_position
813   Jaylon Jones  00-0037106  JoneJa12  CHI         DB                   CB
1042  Jaylon Jones  00-0038407  JoneJa13  IND         DB                   CB

Byron Young:
        full_name     gsis_id    pfr_id team position_x depth_chart_position
1198  Byron Young  00-0038978  YounBy00  PHI         DL                   DT
1243  Byron Young  00-0039137  YounBy01   LA         LB                  OLB


In [252]:
class PositionScorer(ABC):
	"""
	Scores players based on weighted Z-scores, normalized to a 0-100 scale.
    Includes Confidence weighting to handle low-sample players.
	"""

	# 50 is typically "Replacement Level" or "Average Backup" in 0-100 scales
	BASELINE_SCORE = 50.0 

	def __init__(self, position: list[str]):
		self.position = position

	@abstractmethod
	def compute_metrics(self, row: pd.Series) -> Dict[str, float]:
		"""Extract raw stats from a row."""
		pass

	@abstractmethod
	def get_config(self) -> Dict[str, Dict[str, float]]:
		"""
		Configuration for metrics.
		- 'weight': Relative importance (e.g., 10.0 for huge impact, 1.0 for minor).
		- 'direction': 1 (Higher is Better) or -1 (Lower is Better).
		"""
		pass

	@abstractmethod
	def get_confidence_settings(self) -> Dict[str, float]:
		"""
		Returns metric used for confidence and the threshold for 100% confidence.
		Example: {'metric': 'attempts', 'threshold': 300}
		"""
		pass

	def _calculate_confidence(self, row: pd.Series) -> float:
		"""Returns a 0.0 to 1.0 multiplier based on sample size."""
		settings = self.get_confidence_settings()
		metric_key = settings['metric']
		threshold = settings['threshold']
		
		# Get the volume stat (e.g., attempts, snaps)
		# We access the raw dataframe row (metrics are already computed in score_all usually, 
		# but here we rely on the row passed from score_all or compute_metrics output)
		if metric_key in row:
				val = row[metric_key]
		else:
			# Fallback if checking raw row
			val = 0 
		
		# Sigmoid or Linear? Linear is safer for simplicity.
		# Cap at 1.0 (100%)
		confidence = min(1.0, max(0.0, val / threshold))
		
		# Optional: Penalize extremely low samples heavily (curved fit)
		# Power of 0.5 makes confidence grow faster, Power of 2 makes it grow slower.
		# Let's keep it linear for transparency.
		return confidence

	def _calculate_raw_score(self, row: pd.Series, peer_stats: pd.DataFrame, config: Dict) -> float:
		"""Internal helper to calculate the raw weighted sum."""
		raw_score = 0.0
		
		for metric, conf in config.items():
			val = self.compute_metrics(row).get(metric, 0)
			weight = conf['weight']
			direction = conf['direction']
			
			# Get peer distribution for this specific metric
			# (In production, cache these means/stds to speed up)
			peer_vals = peer_stats.apply(lambda r: self.compute_metrics(r).get(metric, 0), axis=1)
			
			if len(peer_vals) > 1 and np.std(peer_vals) > 0:
				mean = np.mean(peer_vals)
				std = np.std(peer_vals)
				z_score = (val - mean) / std
			else:
				z_score = 0.0
				
			# Add to total: Z-Score * Weight * Direction
			# Example: Bad Throw (Z=2.0) * Weight(5) * Dir(-1) = -10.0 impact
			raw_score += (z_score * weight * direction)
			
		return raw_score

	def score_all(self, df: pd.DataFrame) -> pd.DataFrame:
		"""
		Scores all players and returns detailed breakdown of what adds up to raw_score.
		"""
		# Filter for this position
		peers = df[df['depth_chart_position'].isin(self.position)].copy()

		peers = peers[peers['player_name'].notna()]

		if peers.empty:
			return peers

		config = self.get_config()
		metric_names = list(config.keys())

		# 1. Compute raw metrics for all peers at once
		metrics_df = peers.apply(lambda r: pd.Series(self.compute_metrics(r)), axis=1)
		metrics_df = metrics_df.reindex(columns=metric_names)

		print(f"\nScoring position: {self.position} with {len(metrics_df)} players")

		# 2. Calculate distribution (mean/std) based only on "Qualified" peers 
		# (To prevent backups from dragging down the average for stars)
		conf_settings = self.get_confidence_settings()
		qualifying_threshold = conf_settings['threshold'] * 0.2 # e.g., 20% of full workload
		qualified_peers = metrics_df[metrics_df[conf_settings['metric']] >= qualifying_threshold]

		if qualified_peers.empty:
			qualified_peers = metrics_df # Fallback

		means = qualified_peers.mean(skipna=True)
		stds = qualified_peers.std(ddof=0, skipna=True)

		# 3. Compute Z-Scores for EVERYONE (using qualified means)
		z_scores_df = (metrics_df - means) / stds.replace(0, np.nan)
		z_scores_df = z_scores_df.fillna(0.0)

		# 4. Weighted Z-Score Sum
		weights = pd.Series({m: config[m]['weight'] for m in metric_names})
		directions = pd.Series({m: config[m]['direction'] for m in metric_names})
		contrib_df = z_scores_df.multiply(weights, axis=1).multiply(directions, axis=1)

		peers['z_sum'] = contrib_df.sum(axis=1)

		# 5. Convert Z-Sum to 0-100 Scale (Raw Performance)
		# We normalize specific to the distribution found
		raw_mean = peers['z_sum'].mean()
		raw_std = peers['z_sum'].std()

		if raw_std == 0:
			peers['PerformanceScore'] = 50.0
		else:
			# Center at 75 for elite, 50 for average
			peers['PerformanceScore'] = ((peers['z_sum'] - raw_mean) / raw_std) * 15 + 60

		peers['PerformanceScore'] = peers['PerformanceScore'].clip(0, 100)

		# 6. Calculate Confidence & Apply Shrinkage
		# We need the volume metric from metrics_df to calculate confidence
		peers['siq_confidence'] = metrics_df.apply(lambda r: self._calculate_confidence(r), axis=1)

		# FORMULA: Score = (Perf * Conf) + (50 * (1-Conf))
		peers['SportsIQ'] = (peers['PerformanceScore'] * peers['siq_confidence']) + \
							(self.BASELINE_SCORE * (1 - peers['siq_confidence']))

		# Formatting for display
		peers['SportsIQ'] = peers['SportsIQ'].round(1)
		peers['siq_confidence'] = (peers['siq_confidence'] * 100).round(1) # Display as 0-100

		# Attach breakdown
		result_cols = (
			['player_name', 'gsis_id', 'depth_chart_position', 'SportsIQ', 'siq_confidence', 'PerformanceScore']
			+ [f'metric_{m}' for m in metric_names]
		)

		# Helper to join metrics back for CSV output
		peers = peers.join(metrics_df.add_prefix('metric_'))

		return peers[[c for c in result_cols if c in peers.columns]]


In [253]:
# QB Scorer

class QBScorer(PositionScorer):
	def __init__(self): super().__init__(["QB"])
	
	def get_confidence_settings(self):
		return { 'metric': 'attempts', 'threshold': 200 }

	def compute_metrics(self, row):
		atts = max(1, row.get('pass_attempts', 1))
		snaps = max(1, row.get('offense_snaps', 1))
		games_played = max(1, row.get('games_played', 1))
		return {
			'epa_per_play': row.get('passing_epa', 0), # EPA is already per-play usually
			'cpoe': row.get('passing_cpoe', 0),
			'bad_throw_pct': row.get('bad_throw_pct', 0),
			'sack_pct': row.get('sacks_suffered', 0) / snaps,
			'interception_pct': row.get('passing_interceptions', 0) / atts,
			'air_yards': row.get('intended_air_yards_per_pass_attempt', 0),
			'attempts': atts,
			'scrambles': row.get('scramble_yards_per_attempt', 0),
			'pacr': row.get('pacr', 0) ,
			'passing_td_per_game': row.get('passing_tds', 0) / games_played,
		}

	def get_config(self):
		return {
			'epa_per_play':         {'weight': 10.0, 'direction': 1},   
			'cpoe':                 {'weight': 7.0,  'direction': 1},   
			'bad_throw_pct':        {'weight': 7.0,  'direction': -1},  
			'interception_pct':     {'weight': 6.0,  'direction': -1},  
			'sack_pct':             {'weight': 5.0,  'direction': -1},  
			'attempts':             {'weight': 5.0,  'direction': 1},   # Keep (volume) 
			'passing_td_per_game':  {'weight': 4.0,  'direction': 1},  
			'pacr':                 {'weight': 4.0,  'direction': 1},   
			'air_yards':            {'weight': 3.0,  'direction': 1},   
			'scrambles':            {'weight': 2.0,  'direction': 1},   
		}

In [254]:
# RB Scorer 
    
class RBScorer(PositionScorer):
	def __init__(self): super().__init__(["RB"])
	
	def get_confidence_settings(self):
		return {
			'metric': 'carries',
			'threshold': 150
		}

	def compute_metrics(self, row):
		carries = max(1, row.get('carries', 1))
		games_played = max(1, row.get('games_played', 1))	
		return {
			'rushing_epa': row.get('rushing_epa', 0), # EPA is already per-play usually
			'receiving_epa': row.get('receiving_epa', 0),
			'yac_att': row.get('yac_att', 0),
			'rushing_yards': row.get('rushing_yards', 0),
			'receiving_yards': row.get('receiving_yards', 0),
			'rushing_first_downs': row.get('rushing_first_downs', 0),
			'rushing_tds': row.get('rushing_tds', 0),
			'broken_tackles': row.get('brk_tkl', 0),
			'receiving_yards_after_catch': row.get('receiving_yards_after_catch', 0),
			'fumbles': row.get('rushing_fumbles', 0),
			'drop_percent': row.get('drop_percent', 0),
			'carries': carries,
			'carries_per_game': carries / games_played
		}

	def get_config(self):
		return {
			# --- Elite Efficiency (The "Talent" Metrics) ---
			# EPA measures actual value added. Crucial for separating empty yards from impactful ones.
			'rushing_epa':          {'weight': 8.0, 'direction': 1},
			'receiving_epa':        {'weight': 6.0, 'direction': 1},
			
			# Yards After Contact per Attempt (yac_att). 
			# This is the single best stat for isolating the RB's skill from the O-Line's blocking.
			'yac_att':              {'weight': 7.0, 'direction': 1},

			# --- Production & Volume (The "Workhorse" Metrics) ---
			# We value yards, but weight them slightly lower than EPA to avoid just rewarding pure volume.
			'rushing_yards':        {'weight': 6.0, 'direction': 1},
			'receiving_yards':      {'weight': 4.0, 'direction': 1},
			
			# Moving the chains is a key skill for a starter.
			'rushing_first_downs':  {'weight': 5.0, 'direction': 1},
			'rushing_tds':          {'weight': 3.0, 'direction': 1},

			# --- Playmaking & Skill ---
			# Broken Tackles (broken_tackles) shows elusiveness and power.
			'broken_tackles':              {'weight': 4.0, 'direction': 1},
			# Yards After Catch (receiving_yards_after_catch) rewards RBs who turn checkdowns into gains.
			'receiving_yards_after_catch': {'weight': 3.0, 'direction': 1},

			# --- Negatives ---
			# Fumbles are the quickest way to lose a job.
			'fumbles':     			{'weight': 5.0, 'direction': -1},
			# Drops in the passing game kill drive momentum.
			'drop_percent':         {'weight': 2.5, 'direction': -1},
			
			# --- Filter / Bonus ---
			# You might add a small weight to 'carries_per_game' if you specifically want to find high-volume starters.
			'carries'	:                 		{'weight': 0, 'direction': 1},
			'carries_per_game':              {'weight': 2.0, 'direction': 1}
		}    


In [255]:
# WR Scorer 
    
class WRScorer(PositionScorer):
	def __init__(self): super().__init__(["WR"])
	
	def get_confidence_settings(self):
		return {
			'metric': 'targets',
			'threshold': 50
		}

	def compute_metrics(self, row):
		targets = max(1, row.get('targets', 1))
		return {
			'receiving_epa': row.get('receiving_epa', 0),
			'yac_r': row.get('yac_r', 0),  # YAC per reception

			'air_yards_share': row.get('air_yards_share', 0),
        	'wopr': row.get('wopr', 0),
			'receiving_yards': row.get('receiving_yards', 0),
			'receiving_tds': row.get('receiving_tds', 0),
			'receiving_first_downs': row.get('receiving_first_downs', 0),
			'targets': targets,

			'racr': row.get('racr', 0),  # Air yard conversion

			# Route running
        	'adot': row.get('adot', 0),  # Average depth of target

			# Playmaking
			'broken_tackles': row.get('brk_tkl_rec', 0),
			'yac_total': row.get('receiving_yards_after_catch', 0),
			
			# Negatives
			'drop_percent': row.get('drop_percent', 0),
			'fumbles': row.get('receiving_fumbles', 0),
		}

	def get_config(self):
		return {
			# --- Elite Efficiency (The "Talent" Metrics) ---
			# EPA measures actual value added. Crucial for separating empty yards from impactful ones.
			'receiving_epa':          {'weight': 8.0, 'direction': 1},
			
			# Yards After Catch per Reception (yac_r). 
			# This is the single best stat for isolating the WR's skill from the QB's passing.
			'yac_r':              {'weight': 7.0, 'direction': 1},

			# --- Production & Volume (The "Workhorse" Metrics) ---
			# We value yards, but weight them slightly lower than EPA to avoid just rewarding pure volume.
			'receiving_yards':        {'weight': 6.0, 'direction': 1},
			
			# Moving the chains is a key skill for a starter.
			'receiving_first_downs':  {'weight': 5.0, 'direction': 1},
			'receiving_tds':          {'weight': 3.0, 'direction': 1},

			# --- Playmaking & Skill ---
			# Air Yards Share shows ability to get open downfield.
			'air_yards_share':              {'weight': 5.0, 'direction': 1},
			'wopr':                         {'weight': 4.0, 'direction': 1},

			# Route running
			'adot':                         {'weight': 3.0, 'direction': -1},  # Lower is better

			# Playmaking
			'broken_tackles':              {'weight': 4.0, 'direction': 1},
			'yac_total':                   {'weight': 3.0, 'direction': 1},
			
			# --- Negatives ---
			# Drops in the passing game kill drive momentum.
			'drop_percent':         {'weight': 5.0, 'direction': -1},
			# Fumbles are the quickest way to lose a job.
			'fumbles':     			{'weight': 4.0, 'direction': -1},
			
			# --- Filter / Bonus ---
			'targets':              {'weight': 2.0, 'direction': 1}
		}    


In [256]:
# TE Scorer 
    
class TEScorer(PositionScorer):
	def __init__(self): super().__init__(["TE"])
	
	def get_confidence_settings(self):
		return {
			'metric': 'targets',
			'threshold': 40
		}

	def compute_metrics(self, row):
		targets = max(1, row.get('targets', 1))
		return {
			'receiving_epa': row.get('receiving_epa', 0),
			'yac_r': row.get('yac_r', 0),  # YAC per reception

			'air_yards_share': row.get('air_yards_share', 0),
        	'wopr': row.get('wopr', 0),
			'receiving_yards': row.get('receiving_yards', 0),
			'receiving_tds': row.get('receiving_tds', 0),
			'receiving_first_downs': row.get('receiving_first_downs', 0),
			'targets': targets,

			'racr': row.get('racr', 0),  # Air yard conversion

			# Route running
        	'adot': row.get('adot', 0),  # Average depth of target
			'snaps_per_game': row.get('offense_snaps', 0) / max(1, row.get('games_played', 1)),

			# Playmaking
			'broken_tackles': row.get('brk_tkl_rec', 0),
			'yac_total': row.get('receiving_yards_after_catch', 0),
			
			# Negatives
			'drop_percent': row.get('drop_percent', 0),
			'fumbles': row.get('receiving_fumbles', 0),
		}

	def get_config(self):
		return {
			# --- Elite Efficiency (The "Talent" Metrics) ---
			# EPA measures actual value added. Crucial for separating empty yards from impactful ones.
			'receiving_epa':          {'weight': 9.0, 'direction': 1},
			
			# Yards After Catch per Reception (yac_r). 
			# This is the single best stat for isolating the WR's skill from the QB's passing.
			'yac_r':              {'weight': 8.0, 'direction': 1},

			# --- Production & Volume (The "Workhorse" Metrics) ---
			# We value yards, but weight them slightly lower than EPA to avoid just rewarding pure volume.
			'receiving_yards':        {'weight': 6.0, 'direction': 1},
			
			# Moving the chains is a key skill for a starter.
			'receiving_first_downs':  {'weight': 7.0, 'direction': 1},
			'receiving_tds':          {'weight': 5.0, 'direction': 1},

			# --- Playmaking & Skill ---
			# Air Yards Share shows ability to get open downfield.
			'air_yards_share':              {'weight': 5.0, 'direction': 1},
			'wopr':                         {'weight': 4.0, 'direction': 1},
			'snaps_per_game':               {'weight': 3.0, 'direction': 1},

			# Route running
			'adot':                         {'weight': 1.5, 'direction': -1},  # Lower is better

			# Playmaking
			'broken_tackles':              {'weight': 5.0, 'direction': 1},
			'yac_total':                   {'weight': 3.0, 'direction': 1},
			
			# --- Negatives ---
			# Drops in the passing game kill drive momentum.
			'drop_percent':         {'weight': 5.0, 'direction': -1},
			# Fumbles are the quickest way to lose a job.
			'fumbles':     			{'weight': 4.0, 'direction': -1},
			
			# --- Filter / Bonus ---
			'targets':              {'weight': 2.0, 'direction': 1}
		}    


In [257]:
class DLScorer(PositionScorer):
	def __init__(self): super().__init__(["DE", "DT", "NT"])
	
	def get_confidence_settings(self):
		return {
			'metric': 'defense_snaps',
			'threshold': 300
		}

	def compute_metrics(self, row):
		snaps = max(1, row.get('defense_snaps', 1))
		return {
			'pressures': row.get('prss', 0),
			'hurries': row.get('hrry', 0),
			'pressure_rate': row.get('pressure_rate', 0),
			'sacks': row.get('def_sacks', 0),
			'sack_conversion': row.get('def_sacks', 0) / max(1, row.get('pressures', 1)), # How often do they finish?
			'def_tackles_for_loss': row.get('def_tackles_for_loss', 0),
			'tackles_solo': row.get('def_tackles_solo', 0),
			'tackles_assists': row.get('def_tackles_with_assist', 0),
			'bats': row.get('bats', 0),
			'defense_snaps': snaps,

			'missed_tackle_rate': row.get('m_tkl_percent', 0),
			'penalties': row.get('penalties', 0)
		}

	def get_config(self):
		return {
   # Elite tier - Pass rush is king
			'pressure_rate':         {'weight': 10.0, 'direction': 1},
			'sack_conversion':       {'weight': 8.0,  'direction': 1},
			'sacks':                 {'weight': 7.0,  'direction': 1},
			'def_tackles_for_loss':  {'weight': 6.0,  'direction': 1},
   # Secondary tier - Overall disruption
			'pressures':             {'weight': 5.0,  'direction': 1},
			'hurries':               {'weight': 4.0,  'direction': 1},
			'tackles_solo':          {'weight': 3.0,  'direction': 1},
			'tackles_assists':       {'weight': 2.0,  'direction': 1},
			'bats':                  {'weight': 2.0,  'direction': 1},
			'defense_snaps':         {'weight': 2.0,  'direction': 1},
   # Negatives
			'missed_tackle_rate':    {'weight': 7.0,  'direction': -1},
			'penalties':             {'weight': 5.0,  'direction': -1},
		}


In [258]:
class LBScorer(PositionScorer):
	def __init__(self): super().__init__(["LB", 'ILB', 'OLB'])
	
	def get_confidence_settings(self):
		return {
			'metric': 'defense_snaps',
			'threshold': 300
		}

	def compute_metrics(self, row):
		snaps = max(1, row.get('defense_snaps', 1))
		return {
			'tackles_solo': row.get('def_tackles_solo', 0),
			'tackles_assists': row.get('def_tackles_with_assist', 0),
			'tackles_for_loss': row.get('def_tackles_for_loss', 0),
			'm_tkl': row.get('m_tkl', 0),

			'sacks': row.get('def_sacks', 0),
			'qb_hits': row.get('def_qb_hits', 0),
			'fumbles_forced': row.get('def_fumbles_forced', 0),
			
			'pressures': row.get('prss', 0),
			'hurries': row.get('hrry', 0),

			'rat': row.get('rat', 0),  # Passer Rating Against When Targeted
			'pass_defended': row.get('def_passes_defended', 0),
			'interceptions': row.get('def_interceptions', 0),

			'defense_snaps': snaps,
		}

	def get_config(self):
		return {
			'tackles_for_loss':  {'weight': 8.0, 'direction': 1},
			'tackles_solo':      {'weight': 6.0, 'direction': 1},
			'm_tkl':             {'weight': 6.0, 'direction': -1},
			'sacks':             {'weight': 5.0, 'direction': 1},
			'qb_hits':           {'weight': 5.0, 'direction': 1},
			'rat':               {'weight': 5.0, 'direction': -1},
			'fumbles_forced':    {'weight': 5.0, 'direction': 1},
			'interceptions':     {'weight': 4.0, 'direction': 1},
			'pass_defended':     {'weight': 4.0, 'direction': 1},
			'pressures':         {'weight': 3.0,  'direction': 1},
			'hurries':           {'weight': 2.5,  'direction': 1},
			'tackles_assists': 	 {'weight': 2.5, 'direction': 1},
			'defense_snaps':     {'weight': 2.0, 'direction': 1 }
		}


In [259]:
class DBScorer(PositionScorer):
	def __init__(self): super().__init__(["CB", 'FS', 'SS'])
	
	def get_confidence_settings(self):
		return {
			'metric': 'defense_snaps',
			'threshold': 300
		}

	def compute_metrics(self, row):
		snaps = max(1, row.get('defense_snaps', 1))
		return {
			'pass_defended': row.get('def_pass_defended', 0),
			'cmp_pct_allowed': row.get('cmp_percent', 0),
			'rat': row.get('rat', 0),  # Passer Rating Against When Targeted
			'yds_tgt': row.get('yds_tgt', 0),
			'interceptions': row.get('def_interceptions', 0),
			'def_tds': row.get('def_tds', 0),

			'tackles_solo': row.get('def_tackles_solo', 0),
			'm_tkl': row.get('m_tkl', 0),

			'sacks': row.get('def_sacks', 0),
			'td': row.get('td', 0),
			'defense_snaps': snaps,
			'penalties': row.get('penalties', 0)
		}

	def get_config(self):
		return {
			# --- Lockdown Coverage (Primary) ---
			# Passes Defended is the most stable metric for good DB play.
			'pass_defended':     {'weight': 9.0, 'direction': 1},
			# Completion % Allowed. If < 50%, they are elite.
			'cmp_pct_allowed':           {'weight': 8.0, 'direction': -1}, # Lower is better
			# Passer Rating Allowed when targeted.
			'rat':                   {'weight': 6.0, 'direction': -1}, # Lower is better
			# Yards allowed per target.
			'yds_tgt':               {'weight': 5.0, 'direction': -1}, # Lower is better

			# --- Ball Hawking (Game Changing) ---
			'interceptions':     	{'weight': 7.0, 'direction': 1},
			'def_tds':              {'weight': 2.0, 'direction': 1}, # Bonus for Pick-6s

			# --- Tackling / Support (Secondary) ---
			# Important for Safeties, less for Corners.
			'tackles_solo':      {'weight': 4.0, 'direction': 1},
			'm_tkl':                 {'weight': 4.0, 'direction': -1}, # Missed tackles hurt DBs badly in open field
			'sacks':                 {'weight': 2.0, 'direction': 1},
			'defense_snaps':        {'weight': 2.0, 'direction': 1},
			'penalties':             {'weight': 4.0, 'direction': -1},
			
			# --- Penalties (Discipline) ---
			# 'td' here likely refers to TDs allowed in coverage
			'td':                    {'weight': 5.0, 'direction': -1} # TDs allowed
		}


In [260]:
position_mapper = {
	'QB': QBScorer(),
	'RB': RBScorer(),
	'WR': WRScorer(),
	'TE': TEScorer(),
	'DL': DLScorer(),
	'LB': LBScorer(),
	'DB': DBScorer(),
}

for pos, scorer in position_mapper.items():
	print(f"\n\n=== Scoring position: {pos} ===")
	scored_df = scorer.score_all(df_master)
	scored_df.sort_values('SportsIQ', ascending=False).to_csv(f'nfl_{pos.lower()}_sportsiq_{season}.csv', index=False)



=== Scoring position: QB ===

Scoring position: ['QB'] with 81 players


=== Scoring position: RB ===

Scoring position: ['RB'] with 144 players


=== Scoring position: WR ===

Scoring position: ['WR'] with 238 players


=== Scoring position: TE ===

Scoring position: ['TE'] with 135 players


=== Scoring position: DL ===

Scoring position: ['DE', 'DT', 'NT'] with 311 players


=== Scoring position: LB ===

Scoring position: ['LB', 'ILB', 'OLB'] with 245 players


=== Scoring position: DB ===

Scoring position: ['CB', 'FS', 'SS'] with 411 players
